In [1]:
import os
import re
import glob
import feather
import numpy as np
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt
from itertools import product
from functools import partial
from pathlib import Path

# dask
import dask
import dask.dataframe as dd
from dask.distributed import Client

from src.clean_data import clean_rfc_fantasma, clean_sancionados, clean_rupc

# features generales
from src.features_general import (
    monto_por_unidad_compradora,
    proveedores_distintos,
    procedimientos_distintos,
    numero_de_contratos,
)

# features competencia
from src.features_competencia import (
    contratos_por_proveedor,
    porcentaje_procedimientos_por_tipo,
    porcentaje_monto_tipo_procedimiento,
    importe_promedio_por_contrato,
    calcular_IHH_ID_contratos,
    calcular_IHH_ID_monto,
    tendencia_adjudicacion_directa,
    # participantes
    porcentaje_licitaciones_con_un_participante,
    procedimientos_promedio_por_participantes,
    indice_participacion,
    procedimientos_por_participantes_unicos,
    tendencia_incremento_participantes,
    c4_monto_total,
    c4_num_procedimientos,
)

# features transparencia
from src.features_transparencia import (
    # procedimientos
    porcentaje_procedimientos_presenciales,
    contratos_promedio_por_procedimimento,
    # contratos_por_duracion,
    # monto_por_duracion,
    promedio_datos_faltantes_poc_contrato,
    # scraper
    porcentaje_procs_sin_contrato,
    porcentaje_procs_sin_fallo,
    porcentaje_procs_sin_apertura,
    porcentaje_procs_sin_junta_aclaracion,
    porcentaje_procedimientos_sin_archivos,
    promedio_procs_por_archivo,
    tendencia_no_publicacion_contratos,
    porcentaje_adjudicaciones_incompletas,
    porcentaje_invitaciones_incompletas,
    porcentaje_licitaciones_incompletas,
)

# features anomalias
from src.features_anomalias import (
    # procedimientos
    interaccion_rfc_fantasma,
    interaccion_sancionados,
    porcentaje_contratos_por_convenio,
    pc_licitaciones_nacionales_menor_15_dias,
    pc_licitaciones_internacionales_menor_20_dias,
    pc_licitaciones_internacionales_menor_40_dias,
    diferente_estratificacion,
    porcentaje_adjudicaciones_excedieron_monto,
    porcentaje_invitaciones_excedieron_monto,
    # scraper
    promedio_convenios_por_proc,
    porcentaje_procs_sin_convocatoria,
)


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config IPCompleter.use_jedi = False
%load_ext autoreload

%autoreload 2

client = Client()
client

Client Scheduler: tcp://127.0.0.1:36771 Dashboard: http://127.0.0.1:8787,Cluster Workers: 8 Cores: 8 Memory: 16.69 GB


### Procedimientos

In [2]:
df_procedimientos = feather.read_dataframe(
    '../data/bases/procedimientos.feather', nthreads=8)
print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.GOBIERNO == 'APF', :]
df_procedimientos = df_procedimientos.drop('GOBIERNO', axis=1)

df_procedimientos = df_procedimientos.loc[~df_procedimientos.PROVEEDOR_CONTRATISTA.isnull()]
print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.CLAVEUC != 'MISSING']

# Rellenar el caracter faltante
caracter = df_procedimientos.CARACTER.mask(df_procedimientos.CARACTER == '').fillna('SIN REPORTAR')
df_procedimientos = df_procedimientos.assign(CARACTER=caracter)

mapa_codigos_caracter = {'N': 'NACIONAL', 'I': 'INTERNACIONAL', 'T': 'INTERNACIONAL BAJO TLC'}

codigos_caracter = df_procedimientos.loc[df_procedimientos.CARACTER == 'SIN REPORTAR'].NUMERO_PROCEDIMIENTO.map(
    lambda x: x.split('-')[2][0] if isinstance(x, str) else x
)
codigos_caracter = codigos_caracter.map(lambda x: mapa_codigos_caracter.get(x, 'SIN REPORTAR'))

df_procedimientos.loc[df_procedimientos.CARACTER == 'SIN REPORTAR', 'CARACTER'] = codigos_caracter

del caracter, mapa_codigos_caracter, codigos_caracter
print(df_procedimientos.shape)
df_procedimientos.head()

(994177, 47)
(864620, 46)
(864583, 46)


,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,CLAVEUC_REAL,IMPORTE_PESOS
70,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,MULTISERVICIOS Y ASESORIAS DEL CARIBE,HABILITADO,None,319200.0,NaN,MXN,IA-009JZL032-N1-2012,MIXTA,INVITACION A CUANDO MENOS TRES,165782,SERVICIO DE LIMPIEZA CZM 2012,None,ADQUISICIONES,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,NaT,2011-12-16 00:00:00,2011-12-19,NaT,2012-01-01,2013-02-28,115807,SERVICIO DE LIMPIEZA CZM 2012,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,319200.0
71,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,ALEJANDRO ALARCON JIMON,HABILITADO,None,26097.0,NaN,MXN,AA-009JZL032-N4-2012,PRESENCIAL,ADJUDICACION DIRECTA,165738,MANTENIMIENTO A EXTINTORES CZM,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2012-08-10 15:59:00,2012-08-10 16:05:00,NaT,2012-08-20,2012-08-26,2012-12-31,242370,SERVICIO DE MANTENIMIENTO A EXTINTORES AD CZM ...,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,26097.0
72,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,5578,TRANSPORTE ESPECIALIZADO DE PERSONAL NACIONAL,HABILITADO,MEDIANA,361200.0,NaN,MXN,IA-009JZL032-N2-2012,MIXTA,INVITACION A CUANDO MENOS TRES,148131,SERVICIO DE TRANSPORTE DE PERSONAL DE LA ESTAC...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,2011-12-09 14:51:00,2011-12-16 14:00:00,2011-12-19,NaT,2012-01-01,2013-02-28,116667,SERVICIO DE TRANSPORTE DE PERSONAL CZM 2012,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,361200.0
73,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,COMERCIALIZADORA EL MAHARAJA,HABILITADO,None,334264.0,NaN,MXN,AA-009JZL032-N5-2011,MIXTA,ADJUDICACION DIRECTA,276283,SERVICIO DE COMEDOR DEL PERSONAL DE LA ESTACIO...,None,ADQUISICIONES,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,2011-12-28 12:45:00,2011-12-28 17:00:00,NaT,NaT,2012-01-01,2013-02-28,123933,SERVICIO DE COMEDOR AD CZM 2012,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,334264.0
74,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MEDIANA,nan,COMERCIALIZADORA EL MAHARAJA,HABILITADO,None,285948.0,NaN,MXN,AA-009JZL032-N5-2011,MIXTA,ADJUDICACION DIRECTA,148083,SERVICIO DE COMEDOR PARA EL PERSONAL DE LA EST...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,NaN,NO,2011-12-28 12:45:00,2011-12-28 17:00:00,NaT,2011-12-30,2012-01-01,2012-12-31,123933,SERVICIO DE COMEDOR AD CZM 2012,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,285948.0


### Scraper

In [3]:
cols_scraper = [
    'CODIGO_EXPEDIENTE',
    'archivo_anexos', 'archivo_apertura',
    'archivo_contrato', 'archivo_convocatoria',
    'archivo_fallo', 'archivo_junta',
    'numero_archivos','numero_convenios',
]

df_scraper = pd.read_csv(
    '../data/bases/tabla_scraper_2017_10_29.csv',
    usecols=cols_scraper, dtype={'CODIGO_EXPEDIENTE': str, 'Year': str}
)
print(df_scraper.shape)
df_scraper = df_scraper.loc[:, cols_scraper]

## Join con procedimientos
cols_procs=[
    'CLAVEUC', 'NUMERO_PROCEDIMIENTO', 'CODIGO_EXPEDIENTE',
    'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION'
]

df_contratos = (df_procedimientos.groupby(['CLAVEUC', 'NUMERO_PROCEDIMIENTO',
                                           'CODIGO_EXPEDIENTE', 'TIPO_PROCEDIMIENTO',
                                           'TIPO_CONTRATACION'], as_index=False).CODIGO_CONTRATO.count()
                                 .rename(columns={'CODIGO_CONTRATO': 'numero_contratos'}))
print(df_contratos.shape)
df_fechas = df_procedimientos[['CODIGO_EXPEDIENTE', 'FECHA_INICIO']].drop_duplicates(subset=['CODIGO_EXPEDIENTE'])
df_contratos = pd.merge(df_contratos, df_fechas, on='CODIGO_EXPEDIENTE', how='inner')
print(df_contratos.shape)
df_scraper = pd.merge(df_scraper, df_contratos, on='CODIGO_EXPEDIENTE', how='inner')
del df_contratos, df_fechas
print(df_scraper.shape)
df_scraper.head(3)

(1102004, 9)
(679938, 6)
(679938, 7)
(620189, 15)


,CODIGO_EXPEDIENTE,archivo_anexos,archivo_apertura,archivo_contrato,archivo_convocatoria,archivo_fallo,archivo_junta,numero_archivos,numero_convenios,CLAVEUC,NUMERO_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,numero_contratos,FECHA_INICIO
0,599,0,1,0,0,1,1,58,0,018TOQ054,LO-018TOQ054-T2-2010,LICITACION PUBLICA,OBRA PUBLICA,1,2012-01-29
1,1171,0,1,1,1,1,1,6,1,041A00001,LA-010A00001-N3-2011,LICITACION PUBLICA,SERVICIOS,1,2012-01-01
2,1610,0,0,1,0,0,0,3,0,018TOQ999,IA-018TOQ999-I2-2011,INVITACION A CUANDO MENOS TRES,SERVICIOS,1,2014-03-15


### Participantes

In [4]:
cols_parts = [
    'NUMERO_PROCEDIMIENTO',
    'CODIGO_EXPEDIENTE', 
    'CLAVEUC',
    'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION', 'FORMA_PROCEDIMIENTO',
    'PROVEEDOR_CONTRATISTA',
    'ESTATUS_DE_PROPUESTA',
    'ESTATUS_PARTIDA',
    'ESTATUS_FALLO',
    'PRECIO_TOTAL',
]

df_participantes = dd.read_csv(
    '../data/processed/participantes/participantes_*_*.psv', sep='|', usecols=cols_parts,
    dtype={
        'TIPO_PROCEDIMIENTO': str,
        'TIPO_CONTRATACION': str,
        'FORMA_PROCEDIMIENTO': str,
        'ESTATUS_DE_PROPUESTA': str,
        'PERIODO': str,
        'DESCRIPCION_CUCOP': str,
        'PROVEEDOR_CONTRATISTA': str,
        'CODIGO_EXPEDIENTE': str
    }
)

df_participantes = df_participantes.loc[~df_participantes.PROVEEDOR_CONTRATISTA.isnull()]
df_participantes = df_participantes.loc[~df_participantes.CLAVEUC.isnull()]
df_participantes = df_participantes.loc[:, cols_parts]
df_participantes = df_participantes.assign(
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.fillna('SIN REPORTAR'),
    ESTATUS_DE_PROPUESTA=df_participantes.ESTATUS_DE_PROPUESTA.fillna('SIN REPORTAR'),
)
df_participantes = df_participantes.compute()

# Se saca el valor de TIPO_CONTRATACION del número de procedimiento
claves_procs = {
    'L': 'LICITACION PUBLICA',
    'I': 'INVITACION A CUANDO MENOS TRES',
    'A': 'ADJUDICACION DIRECTA',
    'O': 'LICITACION PUBLICA CON OSD',
    'S': 'ADJUDICACION DIRECTA',
    'P': 'PROYECTO DE CONVOCATORIA',
    'E': 'LICITACION PUBLICA',
    'X': 'SIN REPORTAR',
}
claves_contrs = {
    'A': 'ADQUISICIONES',
    'O': 'OBRA PUBLICA',
    'X': 'SIN REPORTAR'
}

claves = df_participantes.NUMERO_PROCEDIMIENTO.map(lambda x: x.split('-')[0])
procedimientos = claves.map(lambda x: claves_procs.get(x[0], None))
contrataciones = claves.map(lambda x: claves_contrs.get(x[1], None))

df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.mask(
        df_participantes.TIPO_PROCEDIMIENTO == 'SIN PLANTILLA', procedimientos),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.mask(
        df_participantes.TIPO_CONTRATACION == 'SIN REPORTAR', contrataciones)
)

df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.fillna('SIN REPORTAR'),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.fillna('SIN REPORTAR'),
)

del claves, procedimientos, contrataciones, cols_parts

# PROCEDIMIENTOS
col_proc = 'NUMERO_PROCEDIMIENTO'
 
df_procs_aux = feather.read_dataframe(
    '../data/bases/procedimientos.feather', nthreads=7,
    columns=[
        col_proc,
        'TIPO_PROCEDIMIENTO',
        'TIPO_CONTRATACION'
    ]
)
print(df_procs_aux.shape)
df_procs_aux = df_procs_aux.drop_duplicates(subset=[col_proc])
print(df_procs_aux.shape)
df_procs_aux = df_procs_aux.rename(
    columns={'TIPO_CONTRATACION': 'TIPO_CONTRATACION_AUX',
             'TIPO_PROCEDIMIENTO': 'TIPO_PROCEDIMIENTO_AUX'}
)

df_participantes = pd.merge(df_participantes,
                            df_procs_aux,
                            on='NUMERO_PROCEDIMIENTO', how='left')

del df_procs_aux

df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.mask(
        df_participantes.TIPO_PROCEDIMIENTO == 'SIN REPORTAR',
        df_participantes.TIPO_PROCEDIMIENTO_AUX).fillna('SIN REPORTAR'),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.mask(
        df_participantes.TIPO_CONTRATACION == 'SIN REPORTAR',
        df_participantes.TIPO_CONTRATACION_AUX).fillna('SIN REPORTAR'),
)

df_participantes = df_participantes.drop(['TIPO_PROCEDIMIENTO_AUX',
                                          'TIPO_CONTRATACION_AUX'], axis=1)

# Se cambia el valor al normalizado

df_participantes.loc[
    df_participantes.TIPO_PROCEDIMIENTO == 'INVITACION A CUANDO MENOS TRES', 'TIPO_PROCEDIMIENTO'
] = 'INVITACION A CUANDO MENOS TRES'

df_participantes.loc[
    df_participantes.TIPO_CONTRATACION == 'SERVICIO DE OBRA PUBLICA', 'TIPO_CONTRATACION'
] = 'SERVICIOS RELACIONADOS CON LA OP'

print(df_participantes.shape)
df_participantes.head()

(1014554, 3)
(804999, 3)
(5216946, 11)


,NUMERO_PROCEDIMIENTO,CODIGO_EXPEDIENTE,CLAVEUC,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,FORMA_PROCEDIMIENTO,PROVEEDOR_CONTRATISTA,ESTATUS_DE_PROPUESTA,ESTATUS_PARTIDA,ESTATUS_FALLO,PRECIO_TOTAL
0,IO-009000972-N30-2011,93548,009000972,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,PRESENCIAL,CONSTRUCCIONES Y PAVIMENTOS BRUFA,GANADOR,NO ADJUDICADO,GANADOR,8963339.44
1,IO-009000972-N30-2011,93548,009000972,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,PRESENCIAL,CONSORCIO CONSTRUCTOR LOBO,PERDEDOR,NO ADJUDICADO,GANADOR,10253719.14
2,IO-009000972-N30-2011,93548,009000972,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,PRESENCIAL,CONSTRUCCIONES CESAR VALENZUELA VELASCO,PERDEDOR,NO ADJUDICADO,GANADOR,10104415.88
3,LA-006HHG001-N64-2011,97983,006HHG001,LICITACION PUBLICA,ADQUISICIONES,MIXTA,ASESORIA EN COMPUTACION SERVICIOS Y ELECTRONICA,GANADOR,NO ADJUDICADO,GANADOR,1970.00
4,AA-018T4I008-N8-2012,135592,018T4I008,ADJUDICACION DIRECTA,ADQUISICIONES,MIXTA,MEDICAL DEPOT,PERDEDOR,NO ADJUDICADO,GANADOR,9343.80


### RFC y Sancionados

In [5]:
# RFC fantasma
df_fantasma = pd.read_csv(
    '../data/bases/RFC fantasma.csv', parse_dates=['Publicación página SAT definitivos'],
    usecols=['RFC', 'Nombre del Contribuyente', 'Publicación página SAT definitivos'],
    dtype={'RFC': str, 'Nombre del Contribuyente': str}, encoding='iso-8859-1', skiprows=2
)
df_fantasma = df_fantasma.rename(columns={'Nombre del Contribuyente': 'Nombre del Contribuyente'.upper()})
df_fantasma = df_fantasma.loc[df_fantasma.RFC != 'XXXXXXXXXXXX']
df_fantasma = clean_rfc_fantasma(df_fantasma)

# Proveedores sancionados
df_sancionados = pd.read_excel('../data/bases/SancionProveedoresContratistas.xls')
df_sancionados = clean_sancionados(df_sancionados)
# Hay un registro con expediente vacío, de la tabla de procedimientos se observa
# que los contratos los realizó en 2014
df_sancionados.loc[
    df_sancionados.PROVEEDOR_CONTRATISTA == 'ALBA MARIA DE LA ASUNCION HERRASTI FERNANDEZ', 'Expediente'
] = 'XXXX/2014'
df_sancionados = df_sancionados.assign(
    Year=df_sancionados.Expediente.map(
        lambda x: int(x.split('/')[1])
    )
)

# poner el valor predeterminado para las funciones que usan estas tablas
interaccion_rfc_fantasma = partial(interaccion_rfc_fantasma, df_rfc_fantasma=df_fantasma)
interaccion_sancionados = partial(interaccion_sancionados, df_sancionados=df_sancionados)

### Montos máximos

In [6]:
df_montos_maximos = pd.read_csv('../data/bases/Montos_maximos.csv',
                                usecols=['Año', 'Tipo de contratación', ' Adjudicación directa ', ' INV3 '],
                                dtype={'Año': int},
                                encoding='iso-8859-1')
df_montos_maximos = df_montos_maximos.rename(columns={c: c.strip() for c in df_montos_maximos.columns})
df_montos_maximos.loc[:, 'INV3'] = (df_montos_maximos.INV3.str.strip()
                                                          .str.replace(',', '')
                                                          .astype(int))
df_montos_maximos.loc[:, 'Adjudicación directa'] = (df_montos_maximos['Adjudicación directa'].str.strip()
                                                                                             .str.replace(',', '')
                                                                                             .astype(int))
df_montos_maximos.loc[:, 'Tipo de contratación'] = (df_montos_maximos['Tipo de contratación'].str.normalize('NFD')
                                                                                             .str.encode('ascii', 'ignore')
                                                                                             .str.decode('utf-8')
                                                                                             .str.upper())
print(df_montos_maximos.shape)
df_montos_maximos.head()

porcentaje_adjudicaciones_excedieron_monto = partial(
    porcentaje_adjudicaciones_excedieron_monto, df_maximos=df_montos_maximos)
porcentaje_invitaciones_excedieron_monto = partial(
    porcentaje_invitaciones_excedieron_monto, df_maximos=df_montos_maximos)

(30, 4)


# Generación de features

In [7]:
funciones_procedimientos = {
    'general': [
        monto_por_unidad_compradora,
        proveedores_distintos,
        procedimientos_distintos,
        numero_de_contratos,
    ],
    'competencia': [
        contratos_por_proveedor,
        porcentaje_procedimientos_por_tipo,
        porcentaje_monto_tipo_procedimiento,
        importe_promedio_por_contrato,
        calcular_IHH_ID_contratos,
        calcular_IHH_ID_monto,
        tendencia_adjudicacion_directa,
        c4_monto_total,
        c4_num_procedimientos,
        
    ],
    'transparencia': [
        porcentaje_procedimientos_presenciales,
        contratos_promedio_por_procedimimento,
        promedio_datos_faltantes_poc_contrato,
    ],
    'anomalias': [
        interaccion_rfc_fantasma,
        interaccion_sancionados,
        porcentaje_contratos_por_convenio,
        pc_licitaciones_nacionales_menor_15_dias,
        pc_licitaciones_internacionales_menor_20_dias,
        pc_licitaciones_internacionales_menor_40_dias,
        diferente_estratificacion,
        porcentaje_adjudicaciones_excedieron_monto,
        porcentaje_invitaciones_excedieron_monto,
    ]
}


funciones_scraper = {
    'general': [
        
    ],
    'competencia': [
            
    ],
    'transparencia': [
        porcentaje_procs_sin_contrato,
        porcentaje_procs_sin_fallo,
        porcentaje_procs_sin_apertura,
        porcentaje_procs_sin_junta_aclaracion,
        porcentaje_procedimientos_sin_archivos,
        promedio_procs_por_archivo,
        tendencia_no_publicacion_contratos,
        porcentaje_adjudicaciones_incompletas,
        porcentaje_invitaciones_incompletas,
        porcentaje_licitaciones_incompletas,
    ],
    'anomalias': [
        promedio_convenios_por_proc,
        porcentaje_procs_sin_convocatoria,
    ]
    
}


funciones_participantes = {
    'general': [
        
    ],
    'competencia': [
        porcentaje_licitaciones_con_un_participante,
        procedimientos_promedio_por_participantes,
        indice_participacion,
        procedimientos_por_participantes_unicos,
        tendencia_incremento_participantes,
    ],
    'transparencia': [
        
    ],
    'anomalias': [
        
    ]
    
}


conceptos = tuple(funciones_procedimientos.keys())

tipos_contratacion = (
    'ADQUISICIONES',
    'SERVICIOS',
    'OBRA PUBLICA',
    'ARRENDAMIENTOS',
    'SERVICIOS RELACIONADOS CON LA OP'
)

for concepto in conceptos:
    for tipo in tipos_contratacion:
        # sub-tablas
        df_procs_aux = df_procedimientos.loc[
            (df_procedimientos.TIPO_CONTRATACION == tipo)
        ]
        df_scraper_aux = df_scraper.loc[
            (df_scraper.TIPO_CONTRATACION == tipo)
        ]
        df_parts_aux = df_participantes.loc[
            (df_participantes.TIPO_CONTRATACION == tipo)
        ]
        # se agarran las funciones para cada tabla
        functions_procs_in_concept = funciones_procedimientos[concepto]
        functions_scraper_in_concept = funciones_scraper[concepto]
        functions_parts_in_concept = funciones_participantes[concepto]
        print(concepto, tipo)
        # Calculo de features
        df_features_procs = [
            function(df_procs_aux, tipo_contratacion=tipo)
            for function in functions_procs_in_concept
        ]
        df_features_procs = [df.set_index('CLAVEUC')
                             for df in df_features_procs
                             if df is not None]
        df_features_scraper = [
            function(df_scraper_aux, tipo_contratacion=tipo)
            for function in functions_scraper_in_concept
        ]
        df_features_scraper = [df.set_index('CLAVEUC')
                               for df in df_features_scraper
                               if df is not None]
        df_features_parts = [
            function(df_parts_aux, tipo_contratacion=tipo)
            for function in functions_parts_in_concept
        ]
        df_features_parts = [df.set_index('CLAVEUC')
                             for df in df_features_parts
                             if df is not None]
        # print([df.shape for df in df_features_scraper])
        # join de features por tabla en concepto
        # Se asume que procedimientos es la BASE PRINCIPAL
        # Join de tablas en el concetp
        df_features = pd.concat(df_features_procs, join='inner', axis=1)
        df_features = df_features.reset_index()
        if len(df_features_scraper) > 0:
            df_features_scraper = pd.concat(df_features_scraper, join='inner', axis=1)
            df_features_scraper = df_features_scraper.reset_index()
            df_features = pd.merge(df_features, df_features_scraper, on='CLAVEUC', how='left')
        if len(df_features_parts) > 0:
            df_features_parts = pd.concat(df_features_parts, join='inner', axis=1)
            df_features_parts = df_features_parts.reset_index()
            df_features = pd.merge(df_features, df_features_parts, on='CLAVEUC', how='left')
        file_path = os.path.join('../data/conceptos/', concepto, tipo, 'features.csv')
        df_features.to_csv(file_path, index=False, quoting=1, encoding='utf-8')
        print(file_path)
        print('-' * 50)


general ADQUISICIONES
../data/conceptos/general/ADQUISICIONES/features.csv
--------------------------------------------------
general SERVICIOS
../data/conceptos/general/SERVICIOS/features.csv
--------------------------------------------------
general OBRA PUBLICA
../data/conceptos/general/OBRA PUBLICA/features.csv
--------------------------------------------------
general ARRENDAMIENTOS
../data/conceptos/general/ARRENDAMIENTOS/features.csv
--------------------------------------------------
general SERVICIOS RELACIONADOS CON LA OP
../data/conceptos/general/SERVICIOS RELACIONADOS CON LA OP/features.csv
--------------------------------------------------
competencia ADQUISICIONES
../data/conceptos/competencia/ADQUISICIONES/features.csv
--------------------------------------------------
competencia SERVICIOS
../data/conceptos/competencia/SERVICIOS/features.csv
--------------------------------------------------
competencia OBRA PUBLICA
../data/conceptos/competencia/OBRA PUBLICA/features.csv

In [21]:
from compranet_src.features_competencia import procs_por_participantes_unicos

tipo = 'SERVICIOS'
df_procs_aux = df_procedimientos.loc[
    (df_procedimientos.TIPO_CONTRATACION == tipo)
]

df_test = procs_por_participantes_unicos(df_procs_aux)
print(df_test.shape)
df_test.head()

(1548, 2)


,CLAVEUC,procs_por_participantes_unicos
0,002000997,1.000000
1,002000999,1.964143
2,003000997,1.500000
3,004000995,1.360000
4,004000998,1.382051


In [19]:
df_test.procs_promedio_por_participantes.describe()

count    1548.000000
mean        0.948139
std         0.116574
min         0.074257
25%         0.958224
50%         1.000000
75%         1.000000
max         1.000000
Name: procs_promedio_por_participantes, dtype: float64

In [3]:
from src.features_anomalias import pc_rotacion_top_proveedores

tipo = 'SERVICIOS'
df_procs_aux = df_procedimientos.loc[
    (df_procedimientos.TIPO_CONTRATACION == tipo)
]

# df_uno, df_dos = pc_rotacion_top_proveedores(df_procs_aux, 2012, 2016)
df_test = pc_rotacion_top_proveedores(df_procs_aux, 2012, 2016)
df_test.head()

,CLAVEUC,pc_rotacion_top_proveedores,proveedores_first,proveedores_second
0,009JZL032,0.0,{},{}
1,009JZL021,0.0,{},{}
2,019GYR027,0.0,{},{}
3,018TOQ896,1.0,{PAMA COMPUTER},{FRENOS Y REFACCIONES DE JUAREZ}
4,018TQA999,0.0,{},{}


In [71]:
df_test.pc_rotacion_top_proveedores.describe()

count    1548.000000
mean        0.109259
std         0.311542
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: pc_rotacion_top_proveedores, dtype: float64

In [4]:
df_test.sort_values('pc_rotacion_top_proveedores', ascending=False)

,CLAVEUC,pc_rotacion_top_proveedores,proveedores_first,proveedores_second
992,009JZL002,1.0,{SONDA MEXICO},{PROTECCION Y ALARMAS PRIVADAS}
189,007000997,1.0,{DMG MORI SEIKI MEXICO},{INGENIERIA Y DESARROLLO DE OBRAS SUMA}
348,008000968,1.0,"{CONTROL DE SEGURIDAD PRIVADA INDUSTRIAL, ECO ...",{}
967,018TOQ060,1.0,"{MEVE SOLUCIONES, MARENTA TECNOLOGIA Y EDUCACI...",{}
178,027A00999,1.0,"{PULIDO ADMINISTRACIONES Y CONSTRUCCIONES, MAL...",{}
681,008F00997,1.0,"{ADRIANA FELIX DE LA ROSA, MIGUEL RUBEN FLORES...",{JULIAN CURIEL VALLECILLO}
343,006HHE001,1.0,{INDRA SISTEMAS MEXICO},{}
181,009000962,1.0,{ROXANA VILLARREAL CABALLERO},{}
1075,018TOQ806,1.0,{LETICIA BASTIDAS GASTELUM},{}
185,006G1H003,1.0,{CAR INFORMATION SYSTEM},{}


In [75]:
df_procs_aux.loc[df_procs_aux.CLAVEUC == '009JZL002']

,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,CLAVEUC_REAL,IMPORTE_PESOS
127286,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-GERENCIA DE LICITACIONES #009JZL002,009JZL002,MIGUEL ANGEL BUSTOS GUERRERO,NO MIPYME,4980,PROTECCION Y ALARMAS PRIVADAS,HABILITADO,NO MIPYME,224400.00,NaN,MXN,SA-009JZL002-N220-2011,PRESENCIAL,ADJUDICACION DIRECTA,84065,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2011-12-09 12:23:00,2011-12-09 13:00:00,NaT,NaT,2012-01-01,2012-12-31,116595,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,Z15122015 12 ADJUDICACION DIRECTA NACIONAL SIM...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,2012,009JZL002,2.244000e+05
127287,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-GERENCIA DE LICITACIONES #009JZL002,009JZL002,MIGUEL ANGEL BUSTOS GUERRERO,NO MIPYME,4980,PROTECCION Y ALARMAS PRIVADAS,HABILITADO,NO MIPYME,183600.00,NaN,MXN,SA-009JZL002-N220-2011,PRESENCIAL,ADJUDICACION DIRECTA,84063,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2011-12-09 12:23:00,2011-12-09 13:00:00,NaT,NaT,2012-01-01,2012-12-31,116595,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,Z15122015 12 ADJUDICACION DIRECTA NACIONAL SIM...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,2012,009JZL002,1.836000e+05
127288,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-GERENCIA DE LICITACIONES #009JZL002,009JZL002,MIGUEL ANGEL BUSTOS GUERRERO,NO MIPYME,4980,PROTECCION Y ALARMAS PRIVADAS,HABILITADO,NO MIPYME,183600.00,NaN,MXN,SA-009JZL002-N220-2011,PRESENCIAL,ADJUDICACION DIRECTA,84062,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2011-12-09 12:23:00,2011-12-09 13:00:00,NaT,NaT,2012-01-01,2012-12-31,116595,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,Z15122015 12 ADJUDICACION DIRECTA NACIONAL SIM...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,2012,009JZL002,1.836000e+05
127289,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-GERENCIA DE LICITACIONES #009JZL002,009JZL002,MIGUEL ANGEL BUSTOS GUERRERO,NO MIPYME,4980,PROTECCION Y ALARMAS PRIVADAS,HABILITADO,NO MIPYME,215880.00,NaN,MXN,SA-009JZL002-N220-2011,PRESENCIAL,ADJUDICACION DIRECTA,84058,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2011-12-09 12:23:00,2011-12-09 13:00:00,NaT,NaT,2012-01-01,2012-12-31,116595,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,Z15122015 12 ADJUDICACION DIRECTA NACIONAL SIM...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,2012,009JZL002,2.158800e+05
127290,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-GERENCIA DE LICITACIONES #009JZL002,009JZL002,MIGUEL ANGEL BUSTOS GUERRERO,NO MIPYME,4980,PROTECCION Y ALARMAS PRIVADAS,HABILITADO,NO MIPYME,213000.00,NaN,MXN,SA-009JZL002-N220-2011,PRESENCIAL,ADJUDICACION DIRECTA,84056,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2011-12-09 12:23:00,2011-12-09 13:00:00,NaT,NaT,2012-01-01,2012-12-31,116595,CONTRATACION DEL SERVICIO DE SEGURIDAD Y VIGIL...,Z15122015 12 ADJUDICACION DIRECTA NACIONAL SIM...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,No,MX,NaN,None,NaN,2012,009JZL002,2.130000e+05
127291,AEROPUERTOS Y SERVICIOS AUXILIARES,A

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd5770fbef0> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd5770fbda0> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576f86ef0> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576e3f390> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576fb1240> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576e744a8> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576ee2748> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576ee2be0> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

...

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd5770b1b70> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd5770b15c0> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576edeac8> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576edea20> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd684b7e080> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd684b7e940> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd684b88a20> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd684b88358> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd684b1afd0> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x7fd576fdf518> exception was never retrieved: Traceback (most recent call last):
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/site-packages/distributed/process.py", line 36, in _call_and_set_future
    res = func(*args, **kwargs)
  File "/home/raul/miniconda3/envs/compranet/lib/python3.6/multiprocessing/process.py", line 113, in terminate
    self._popen.terminate()
AttributeError: 'None

In [69]:
top_2012 = {1,2,3, 7, 8}
top_2016 = {2,3,4,5}

1 - len(top_2016 & top_2012) / len(top_2012)

0.6

In [59]:
list(df_procedimientos.loc[df_procedimientos.CLAVEUC == '012K00999'].PROVEEDOR_CONTRATISTA.unique())

['FARMACOS ESPECIALIZADOS',
 'AGUILAR MORENO PERFECTO',
 'FORZA TRAVEL',
 'RALCA',
 'RAUL DURAN NAVARRETE',
 'ANUAR ISMAEL LUNA CADENA',
 'MA DEL ROSARIO PADILLA GARCIA',
 'HIPO-PAK',
 'GRUPO FARMACOS ESPECIALIZADOS',
 'FARMACEUTICOS MAYPO',
 'DALTEM PROVEE NACIONAL',
 'DIBITER',
 'MA ELENA RAMOS RODRIGUEZ',
 'MA SANDRA CORINA',
 'SIGALES COMPANIA PAPELERA SA DE V',
 'SERVICIOS DE PERSONAL Y CAPTURA DE DATOS',
 'GRUPO REMIS',
 'TAVATA DISTRIBUCIONES',
 'SERGIO HERNANDEZ MONDRAGON',
 'BENJAMIN IZQUIERDO MOJICA',
 'DISTRIBUIDORA DESC',
 'ISABEL CAMPOS RUIZ',
 'JUAN CARLOS RODRIGUEZ ESPINOSA',
 'ARTURO MEJIA RAMIREZ',
 'EDITH FLORES PEREZ',
 'DENTILAB',
 'MARCO ANTONIO MARTINEZ LASSO',
 'MA GUADALUPE HERNANDEZ GONZALEZ',
 'HELSEK COMPANY',
 'GRUPO ROMAY',
 'GUILLERMO LEON RAMIREZ MARTINEZ',
 'LANDSTEINER PHARMA',
 'SAVI DISTRIBUCIONES',
 'COMERCIALIZADORA DICLINSA',
 'ORLANDO CORNEJO MONTANO',
 'AID ARTE I DISENO',
 'HIPO PAK',
 'JORGE OSWALDO MANZANO PAYAN',
 'CECOFIN SC',
 'JET VAN CAR 

In [61]:
df_procedimientos.loc[df_procedimientos.CLAVEUC == '012K00999'].FECHA_ARCHIVO.value_counts()

2012    47
2017    42
2016    35
2015    34
2013    33
2014    17
Name: FECHA_ARCHIVO, dtype: int64

In [54]:
df_test.proveedores_first.isnull().value_counts()

False    1548
Name: proveedores_first, dtype: int64

In [52]:
df_test.proveedores_first.isnull().value_counts()

False    1076
True      472
Name: proveedores_first, dtype: int64

In [45]:
df_test.sample(n=10)

,CLAVEUC,proveedores_first,proveedores_second
177,011MHL001,"{EMERGENTE PRODUCCIONES SC, DEMOS DESARROLLO D...","{DEMOS DESARROLLO DE MEDIOS, LA CRONICA DIARIA..."
115,020L00993,{},NaN
784,009J2Y001,{},NaN
164,03891Q001,"{TAVERA HERMANOS, DORA AGUILAR TORRES, PUBLIC ...","{AXA SEGUROS, DORA AGUILAR TORRES, TERESA AZUA..."
860,040100992,{},{}
1362,018TOQ107,NaN,NaN
915,018TOQ899,{ABB MEXICO},{ANA MARIA AGUILAR GOMEZ}
605,019GYN009,{EFECTIVALE},{}
1024,009000987,{},{}
356,012NAW001,{},{}


(1548, 3)

In [12]:
df_uno = df_uno.assign(tam=df_uno.proveedores.map(lambda x: len(x)))

In [14]:
df_uno.sort_values('tam', ascending=False)

,CLAVEUC,proveedores,tam
358,011MDE001,"{MANUELL BARRERA Y ASOCIADOS, ARLO ASESORES SC...",30
154,008JAG007,"{EJIDO DE TEQUESQUITENGO, EJIDO DE TEHUIXTLA, ...",11
87,006HKA001,"{MIGUEL ANGEL MARTINEZ CAMACHO, MARCOS JUAN LO...",8
102,008000976,"{EMMANUEL DUENAS ARMENTA, AMELIA PEREZ RODRIGU...",7
130,008F00996,"{ERNESTO LORENZO CASTANEDA PARRA, JOSE PABLO H...",6
509,018T0K001,"{WEATHERFORD DE MEXICO, HIR COMPANIA DE SEGURO...",6
128,008F00994,"{JUAN ANTONIO TRINIDAD AGUILAR, OSCAR DE ZAMAC...",5
653,018TOQ809,"{ASESORIA Y CONSULTORIA MACURTIA SC, CYBERLIFE...",4
125,008F00001,"{ORACLE DE MEXICO, ASOCIACION NACIONAL DE TIEN...",3
1051,03891Q001,"{TAVERA HERMANOS, DORA AGUILAR TORRES, PUBLIC ...",3


In [23]:
df_uno.tam.describe()

count    1076.000000
mean        0.267658
std         1.180821
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        30.000000
Name: tam, dtype: float64

In [21]:
pd.DataFrame(df_dos.CLAVEUC.unique(), columns=['CLAVEUC'])

(1236, 1)

In [16]:
df_dos.shape

(1236, 2)

In [22]:
df_procs_aux.CLAVEUC.nunique()

1548

In [9]:
df_dos.head()

,CLAVEUC,proveedores
0,002000997,{}
1,002000999,{}
2,004000998,{}
3,004000999,{}
4,004D00001,{}


In [13]:
for g, subdf in df_dos.groupby('CLAVEUC'):
    df_aux = subdf.sort_values('pc_monto', ascending=False)
    df_aux = df_aux.assign(pc_monto_cumsum=df_aux.pc_monto.cumsum())

In [24]:
df_prueba = df_aux.loc[df_aux.pc_monto_cumsum <= 0.20]
# n_rows = df_aux.loc[df_aux.pc_monto_cumsum <= 0.20].shape[0]
if df_prueba.shape[0] == 0:
    df_prueba = df_aux.iloc[0:1, :]

In [25]:
df_prueba

,CLAVEUC,PROVEEDOR_CONTRATISTA,pc_monto,pc_monto_cumsum
49393,046D00999,DOWELL SCHLUMBERGER DE MEXICO,0.451374,0.451374


In [14]:
df_aux

,CLAVEUC,PROVEEDOR_CONTRATISTA,pc_monto,pc_monto_cumsum
49393,046D00999,DOWELL SCHLUMBERGER DE MEXICO,0.451374,0.451374
49401,046D00999,L Y D TECHNOLOGIES,0.345926,0.797300
49407,046D00999,WOOD MACKENZIE INC,0.148030,0.945330
49399,046D00999,JORGE JONATHAN TORRES CORONA,0.017024,0.962353
49406,046D00999,VIAJES PREMIER SA,0.008224,0.970577
49403,046D00999,PRICEWATERHOUSECOOPERS SC,0.007664,0.978241
49386,046D00999,ALTOPETRUM & GENERAL OIL DE MEXICO,0.007630,0.985871
49400,046D00999,JOSE CHERNOVETZKY HOLTZ,0.004658,0.990529
49391,046D00999,CONSORCIO LOGISTICO EMPRESARIAL J&A,0.002531,0.993060
49404,046D00999,SIDIMEX TECNOLOGIAS DE INFORMACION,0.002482,0.995542


In [34]:
# tipo = 'SERVICIOS'
# df_procs_aux = df_procedimientos.loc[
#     (df_procedimientos.TIPO_CONTRATACION == tipo)
# ]
# # porcentaje_procedimientos_por_tipo(df_procs_aux)
# porcentaje_monto_tipo_procedimiento(df_procs_aux)

In [33]:
from src.features_competencia import c4_monto_total, c4_num_procedimientos
# df_test = c4_monto_total(df_procs_aux)
df_test = c4_num_procedimientos(df_procs_aux)
df_test.head(10)

,CLAVEUC,c4_num_procedimientos
0,002000997,0.444444
1,002000999,0.085271
2,003000997,1.000000
3,004000995,0.147059
4,004000998,0.066050
5,004000999,0.527273
6,004D00001,0.175439
7,004E0K998,0.154930
8,004E2D001,0.277523
9,004EZN999,0.145038


In [32]:
df_test.loc[df_test.CLAVEUC == '002000999']

,CLAVEUC,c4_monto_total
1,002000999,0.608329


In [25]:
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite
from sklearn.linear_model import Ridge


df_test = tendencia_incremento_participantes(df_parts_aux)

In [ ]:
# df_algo = df_participantes.loc[df_participantes.TIPO_CONTRATACION == 'SERVICIOS']
# tendencia_incremento_participantes(df_algo)

In [3]:
df_participantes.TIPO_CONTRATACION.value_counts(dropna=False)

ADQUISICIONES                       4044473
SERVICIOS RELACIONADOS CON LA OP     516361
SERVICIOS                            489500
OBRA PUBLICA                         138923
ARRENDAMIENTOS                        26794
SIN REPORTAR                            895
Name: TIPO_CONTRATACION, dtype: int64